#MUSIC RECOMMENDER BOT

By:
  - Eduardo Ruíz Martínez - 219453
  - Óscar Delgado Rueda - 229935

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
%cd /content/drive/MyDrive/4t/Tecniques de Processament de Llenguatge/FINAL_LAB

/content/drive/MyDrive/4t/Tecniques de Processament de Llenguatge/FINAL_LAB


In [34]:
#Print path and content of the current directory
!pwd
!ls

/content/drive/MyDrive/4t/Tecniques de Processament de Llenguatge/FINAL_LAB
 data  'Final Report.gdoc'   lib   Music_Recommender_Bot.ipynb	 Report.gdoc


In [35]:
import numpy as np
from pathlib import Path
import tensorflow
import keras
import nltk
from sklearn.pipeline import Pipeline
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

from numpy import dot
from numpy.linalg import norm

import re

nltk.download('punkt')
!pip install transformers

!pip install json-lines

import json_lines
import random

!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#DATASET IMPORT

In [36]:
f = open('data/data/genres.jl', 'rb')

#Labels for genres
genre_labels = []

for elem in json_lines.reader(f):
  genre_labels.append(elem['name'])

#Labels for moods
mood_labels = ['happy', 'sad', 'focused', 'angry', 'relaxed']

#Labels for dates
date_labels = ['1970', '1980', '1990', '2000', '2010', '2020']

#another way to reffer to dates, but with words. This is not stored in data, it is used in get_date_from_sentence() method
new_date_keywords = ['modern', 'new']
old_date_keywords = ['old', 'older']

#Labels for artist
artist_labels = []

f = open('data/data/tags.jl')

#Data where each element has the form {'genre':_,'artist':_,'track':_,'position':_}
data = []

for elem in json_lines.reader(f):
  data.append({'genre':elem['genre'],
               'artist':elem['artist'],
               'track':elem['track'],
               'position':int(elem['position']),
               'mood':random.choice(mood_labels),
               'date':random.choice(date_labels)})
  
  artist_labels.append(elem['artist'].lower())

artist_labels = [*set(artist_labels)]

#Banned artists
artist_labels.remove("can")
artist_labels.remove("you")
artist_labels.remove("do")
artist_labels.remove("else")
artist_labels.remove("more")
artist_labels.remove("songs")
artist_labels.remove("focused")
artist_labels.remove("this")
artist_labels.remove("1980")
artist_labels.remove("hello")

In [37]:
print(data[1])
print(artist_labels[:100])

{'genre': 'pop rap', 'artist': 'B.o.B, Hayley Williams', 'track': 'Airplanes (feat. Hayley Williams of Paramore)', 'position': 1, 'mood': 'happy', 'date': '1970'}
['rossington', 'noteworthy', 'kanaku y el tigre', 'kramus', 'devnik', 'miller/tysen, company', 'n.y.a.s.', 'stl', 'danny darko, hannah young', 'ricardo castro, jorge federico osorio', 'peter mergener & klaus hoffmann-hoock', 'bob dufford, s.j.', 'king wasiu ayinde marshal i', 'upper', 'laineen kasperi, palava kaupunki, ismo alanko', 'gym class heroes, adam levine', 'funktomas', 'david sylvian', 'max pashm', 'pleasure p', 'eversham', 'brisbane ballroom dance combo', 'fiesta 85', 'zebre', 'ao cubo', 'katharine whalen', 'kari rueslåtten', 'linda kiraly, pras michel, rh3', 'towa tei, taprikk sweezee', 'edith barr, eva ayllón', 'julian morris', 'mike mainieri, nils petter molvær, bugge wesseltoft, eivand aarset, lars danielsson, anders engen, paolo vinaccia, jan bang, dj strangefruit', 'susanu', 'oslo beurs', 'twin steps', 'adam b

#Transformer

In [38]:
transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

#Functions

In [39]:
embedding_dim = 100

def remove_white_space(text):
    return ' '.join(text.split())

def sentence_detection(text):
    text = text.replace('\\n', '')
    text = remove_white_space(text)
    text = text.lower()
    sentence = nltk.sent_tokenize(text)
    
    #text = " ".join(sentence)
    return sentence

def sent_to_word_tokenizer(text):
    words = []
    for sentence in text:
      #if(sentence != (symbols)):
      tokenizer = RegexpTokenizer(r'\w+')
      words.append(tokenizer.tokenize(sentence))

    return words
    
def get_embedding(text):
  return transformer_model.encode(text)

def cos(x,y):
  return dot(x, y)/(norm(x)*norm(y))


#POSSIBLE QUESTIONS FROM THE USER

In [40]:
#Question labels indexes
TOP_ARTISTS_FROM_GENRE = 0
TOP_TRACKS_FROM_GENRE = 1
SONG_FROM_MOOD = 2
ARTIST_FROM_MOOD = 3
SONG_FROM_DATE = 4
ARTIST_FROM_DATE = 5
SIMILAR_ARTIST = 6
NOT_VALID = 7

#Question labels
question_labels = ['Top-artists in genre question',
                   'Top-tracks in genre question',
                   'Song from mood question',
                   'Artist from mood question',
                   'Song from date question',
                   'Artist from date question',
                   'Similar artist question',
                   'Not valid']

In [41]:
questions = {
    
    #'Top-artists in genre question'
    'Can you tell me which bands are the most popular in _?':'Top-artists in genre question',
    'Which are the best bands in _?':'Top-artists in genre question',
    'Do you know any _ popular bands?':'Top-artists in genre question',
    'Do you know any _ famous bands?':'Top-artists in genre question',
    'Which bands are the best in _?':'Top-artists in genre question',

    'Best artists in _':'Top-artists in genre question',
    'Most well known artists in _':'Top-artists in genre question',
    'Most well known bands in _':'Top-artists in genre question',
    'Top artists in _':'Top-artists in genre question',
    'Top bands in _':'Top-artists in genre question',
    'Which are the top bands in _?':'Top-artists in genre question',
    'Which are the top artists in _?':'Top-artists in genre question',
    'Which are the most listened bands in _?':'Top-artists in genre question',
    'Which are the most listened artists in _?':'Top-artists in genre question',
    'Most listened artists in _':'Top-artists in genre question',
    'Most listened bands in _':'Top-artists in genre question',

    'Can you tell me the most popular artists in _?':'Top-artists in genre question',
    'Who are the most relevant artists in _?':'Top-artists in genre question',
    'Do you know any _ popular artists?':'Top-artists in genre question',
    'Do you know any _ famous artists?':'Top-artists in genre question',

    #'Top-tracks in genre question'

    'Can you tell me the most popular tracks in _?':'Top-tracks in genre question',
    'Which are the most relevant tracks in _?':'Top-tracks in genre question',
    'Do you know any _ popular tracks?':'Top-tracks in genre question',
    'Do you know any _ famous tracks?':'Top-tracks in genre question',
    'Can you tell me the most popular songs in _?':'Top-tracks in genre question',
    'Which are the most relevant songs in _?':'Top-tracks in genre question',
    'Do you know any _ popular songs?':'Top-tracks in genre question',
    'Do you know any _ famous songs?':'Top-tracks in genre question',

    'Which are the most popular tracks in _?':'Top-tracks in genre question',
    'Which are the most popular songs in _?':'Top-tracks in genre question',
    'The best _ songs':'Top-tracks in genre question',
    'The best _ tracks':'Top-tracks in genre question',
    'Most listened _ songs':'Top-tracks in genre question',
    'Most listened _ tracks':'Top-tracks in genre question',
    'Any well known _ songs':'Top-tracks in genre question',
    'Any well known _ tracks':'Top-tracks in genre question',

    #'Song from mood question'

    'Do you know any song that makes me feel _?':'Song from mood question',
    'Can you tell me a song that makes me feel _?':'Song from mood question',
    'Do you know any _ songs?':'Song from mood question',
    'Im _, do you recommend me any song?':'Song from mood question',
    'Do you know any song for _?':'Song from mood question',
    'I feel a bit _, can you recommend me a song?':'Song from mood question',
    'What song I should listen to if I feel _?':'Song from mood question',
    'Today I feel _, what song should I listen to?':'Song from mood question',
    'Im feeling _ do you have any songs for that?':'Song from mood question',
    'Im feeling _ do you have any tracks for that?':'Song from mood question',
    'Im _ do you have any songs for that?':'Song from mood question',
    'Im _ do you have any tracks for that?':'Song from mood question',

    
    #'Artist from mood question'

    'Do you know any band that makes me feel _?':'Artist from mood question',
    'Can you tell me a band that makes me feel _?':'Artist from mood question',
    'Im _, do you recommend me any band?':'Artist from mood question',
    'Do you know any band for _?':'Artist from mood question',
    'I feel a bit _, can you recommend me a band?':'Artist from mood question',
    'What band I should listen to if I feel _?':'Artist from mood question',
    'Today I feel _, what band should I listen to?':'Artist from mood question',

    'Do you know any artist that makes me feel _?':'Artist from mood question',
    'Can you tell me an artist that makes me feel _?':'Artist from mood question',
    'Im _, do you recommend me any artist?':'Artist from mood question',
    'Do you know any artist for _?':'Artist from mood question',
    'I feel a bit _, can you recommend me an artist?':'Artist from mood question',
    'What artist I should listen to if I feel _?':'Artist from mood question',
    'Today I feel _, what artist should I listen to?':'Artist from mood question',

    #'Song from date question'

    'Do you know any song from the date _?':'Song from date question',
    'Can you tell me a song released in _?':'Song from date question',

    'Do you know any song more modern?':'Artist from date question',
    'Do you know any song more new?':'Artist from date question',
    'Do you know any old song?':'Artist from date question',

    'Do you know any artist from the date _?':'Artist from date question',
    'Can you tell me an artist from the date _?':'Artist from date question',

    'Do you know any band from the date _?':'Artist from date question',
    'Can you tell me a band from the date _?':'Artist from date question',

    'Do you know any artist more modern?':'Artist from date question',
    'Do you know any artist more new?':'Artist from date question',
    'Do you know any old artist?':'Artist from date question',

    'Do you know any band more modern?':'Artist from date question',
    'Do you know any band more new?':'Artist from date question',
    'Do you know any old band?':'Artist from date question',

    'I like this band you recommended to me. Do you know other similar bands?':'Similar artist question',
    'Do you know any band similar to the one you recommended me?':'Similar artist question',
    'Do you know any band similar to _?':'Similar artist question',
    'Can you tell me a band similar to _?':'Similar artist question',
    'I like _. Could you recommend something like that to me?':'Similar artist question'
}

answers = {
    'Top-artists in genre question':['The top-artists in * are:\n_'],
    'Top-tracks in genre question':['The top-tracks in * are:\n_'],
    'Song from mood question':['When I feel *, I like to listen to the track _, from +.'],
    'Artist from mood question':['When I feel *, I like to listen to the artist _.'],
    'Song from date question':['You should listen to _, a track from the artist +, from *.'],
    'Artist from date question':['You should listen to _, an artist from *.'],
    'Similar artist question':['An artist similar to * could be _.'],
    'Not valid':['I don' + "'" + 't understand you. Could you ask me again please?']
}

In [42]:
#This dictionaries are auxiliar dictionaries for better tag recognition in get_type_of_sentence method

questions_genre_related = {}

for key in questions.keys():
    if("genre" in questions[key]):
      questions_genre_related[key] = questions[key]

questions_mood_related = {}

for key in questions.keys():
    if("mood" in questions[key]):
      questions_mood_related[key] = questions[key]

questions_date_related = {}

for key in questions.keys():
    if("date" in questions[key]):
      questions_date_related[key] = questions[key]

questions_artist_related = {}

for key in questions.keys():
    if('Similar artist question' == questions[key]):
      questions_artist_related[key] = questions[key]

def get_auxiliar_dictionary(input):

  #First we look at the current sentence keywords

  if(len(get_artist_from_sentence(input)) > 0):
    return questions_artist_related;

  if(len(get_genre_from_sentence(input)) > 0):
    return questions_genre_related;

  if(len(get_mood_from_sentence(input)) > 0):
    return questions_mood_related;

  if(len(get_date_from_sentence(input)) > 0):
    return questions_date_related;

  #Get dialog history keywords

  if(len(get_dialog_history(PREVIOUS_GENRE)) > 0):
    return questions_artist_related;

  if(len(get_dialog_history(PREVIOUS_MOOD)) > 0):
    return questions_mood_related;

  if(len(get_dialog_history(PREVIOUS_DATE)) > 0):
    return questions_date_related;
    
  return questions

#Dialog history

In [43]:
#Question labels indexes
PREVIOUS_GENRE = 0
PREVIOUS_MOOD = 1
PREVIOUS_DATE = 2

dialog_history = ["","",""]

def reset_dialog_history():
  for i in range(0, len(dialog_history)):
    dialog_history[i] = ""

def get_dialog_history(tag):
  return dialog_history[tag]

def get_all_dialog_history():
  return get_dialog_history(PREVIOUS_GENRE), get_dialog_history(PREVIOUS_MOOD), get_dialog_history(PREVIOUS_DATE)

def update_dialog_history(tag, value):
  dialog_history[tag] = value

def get_and_update_genre(sent):

  result = get_genre_from_sentence(sent)

  if(len(result)>0):
    update_dialog_history(PREVIOUS_GENRE, result)
  else:
    result = get_dialog_history(PREVIOUS_GENRE)

  return result

def get_and_update_mood(sent):

  result = get_mood_from_sentence(sent)

  if(len(result)>0):
    update_dialog_history(PREVIOUS_MOOD, result)
  else:
    result = get_dialog_history(PREVIOUS_MOOD)

  return result

def get_and_update_date(sent):

  result = get_date_from_sentence(sent)

  if(len(result)>0):
    update_dialog_history(PREVIOUS_DATE, result)
  else:
    result = get_dialog_history(PREVIOUS_DATE)

  return result

#Filter data method

In [44]:
def filter_data(tag, value, dataset):
  filtered_data = []
  for elem in dataset:
    if elem[tag] == value:
      filtered_data.append(elem)

  return filtered_data

#Debug functions

In [45]:
def print_keyword_from_sentences(input):
  print("\n--Keywords from CURRENT sentence--")
  print("Genre:", get_genre_from_sentence(input))
  print("Mood:", get_mood_from_sentence(input))
  print("Date:", get_date_from_sentence(input))
  print("Artist:", get_artist_from_sentence(input))

def print_dialog_history():
  print("\n--Dialog history--")
  print("Previous genre:", get_dialog_history(PREVIOUS_GENRE))
  print("Previous mood:", get_dialog_history(PREVIOUS_MOOD))
  print("Previous date:", get_dialog_history(PREVIOUS_DATE))

#Dialog Tag

In [46]:
pip install -U DialogTag

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
from dialog_tag import DialogTag

dialog_tag_model = DialogTag('distilbert-base-uncased')

distilbert-base-uncased found in cache. Loading model...


Some layers from the model checkpoint at /root/.dialog-tag/models/distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /root/.dialog-tag/models/distilbert-base-uncased and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
def predict_tag(text):
  return dialog_tag_model.predict_tag(text)

sentence = "Yes"
output = predict_tag(sentence)
print(output)
# output: 'Statement-non-opinion'

sentence = "No"
output = predict_tag(sentence)
print(output)
# output: 'Wh-Question'

1/1 [==============================] - 3s 3s/step
Acknowledge (Backchannel)
1/1 [==============================] - 0s 57ms/step
No Answers


#CHAT BOT RELATED FUNCTIONS

In [49]:
def sortPos(val):
    return val['position']

def sortDate(val):
    return val['date']

def get_type_of_sentence(input):
  input_emb = get_embedding(input)

  final_dict = get_auxiliar_dictionary(input);

  most_accurate_tag = ""
  max_cosine = 0

  min_score = 0.35

  for key in final_dict.keys():
    question_emb = get_embedding(key)

    curr_cos = cos(input_emb,question_emb)

    if(curr_cos > max_cosine):
      max_cosine = curr_cos
      most_accurate_tag = final_dict[key]

    if(max_cosine < min_score):
      most_accurate_tag = 'Not valid'
  
  print("(Score: " + str(max_cosine) + ")")

  return most_accurate_tag

def get_artist_from_sentence(sent):
  result = ""

  sent = sentence_detection(sent)
  t_sent = sent_to_word_tokenizer(sent)[0]

  for word in t_sent:
    if word in artist_labels:
      result = word

  return result

def get_genre_from_artist(artist):
  result = ""

  for elem in data:
    if elem['artist'].lower() == artist:
      result = elem['genre']

  return result

def get_genre_from_sentence(sent):
  result = ""

  sent = sentence_detection(sent)
  t_sent = sent_to_word_tokenizer(sent)[0]

  for word in t_sent:
    if word in genre_labels:
      result = word

  return result

def get_mood_from_sentence(sent):
  result = ""

  sent = sentence_detection(sent)
  t_sent = sent_to_word_tokenizer(sent)[0]

  for word in t_sent:
    if word in mood_labels:
      result = word

  return result

def get_date_from_keywords(sent):
  result = ""

  sent = sentence_detection(sent)
  t_sent = sent_to_word_tokenizer(sent)[0]

  for word in t_sent:
    if word in new_date_keywords:
      result = date_labels[len(date_labels)-1]
    if word in old_date_keywords:
      result = date_labels[0]
  
  return result

def get_date_from_sentence(sent):
  result = ""

  numbers = re.findall(r'\d+', sent)    # we search numbers

  if len(numbers) == 0:
    result = get_date_from_keywords(sent)
  else:
    for date in date_labels:
      if numbers[0] in date:
        result = date

  return result

#GET TOP FROM GENRE FUNCTIONS

In [50]:
def get_artists_from_genre(sent):
  genre = get_and_update_genre(sent)
  date = get_and_update_date(sent)
  mood = get_and_update_mood(sent)

  artists_from_genre = filter_data('genre', genre, data)

  if(len(date)>0):
    artists_from_genre = filter_data('date', date, artists_from_genre)

  if(len(mood)>0):
    artists_from_genre = filter_data('mood', mood, artists_from_genre)

  artists_from_genre.sort(key=sortPos,reverse=False)
  artists_from_genre = artists_from_genre[:10]

  result = ""
  for i in range(len(artists_from_genre)):
    result = result + "\n" + str(i+1) + "- " + artists_from_genre[i]['artist']

  default_answer = answers['Top-artists in genre question'][0]

  response = default_answer.replace('*', genre)
  response = response.replace('_', result)

  return response

def get_tracks_from_genre(sent):
  genre = get_and_update_genre(sent)
  date = get_and_update_date(sent)
  mood = get_and_update_mood(sent)

  tracks_from_genre = filter_data('genre', genre, data)

  if(len(date)>0):
    tracks_from_genre = filter_data('date', date, tracks_from_genre)

  if(len(mood)>0):
    tracks_from_genre = filter_data('mood', mood, tracks_from_genre)

  tracks_from_genre.sort(key=sortPos,reverse=False)
  tracks_from_genre = tracks_from_genre[:10]

  result = ""
  for i in range(len(tracks_from_genre)):
    result = result + "\n" + str(i+1) + "- " + tracks_from_genre[i]['track']

  default_answer = answers['Top-tracks in genre question'][0]

  response = default_answer.replace('*', genre)
  response = response.replace('_', result)

  return response

#GET ARTIST/SONG FROM MOOD

In [51]:
def get_song_from_mood(sent):
  mood = get_and_update_mood(sent)
  genre = get_and_update_genre(sent)
  date = get_and_update_date(sent)

  tracks_from_mood = filter_data('mood', mood, data)

  if(len(genre)>0):
    tracks_from_mood = filter_data('genre', genre, tracks_from_mood)

  if(len(date)>0):
    tracks_from_mood = filter_data('date', date, tracks_from_mood)

  if(len(tracks_from_mood)>0):
    track_selected = random.choice(tracks_from_mood)

    artist = '"' + track_selected['artist'] + '"'
    result = '"' + track_selected['track'] + '"'

    default_answer = answers['Song from mood question'][0]

    response = default_answer.replace('*', mood)
    response = response.replace('+', artist)
    response = response.replace('_', result)
  else:
    response = ""

  return response

def get_artist_from_mood(sent):
  mood = get_and_update_mood(sent)
  genre = get_and_update_genre(sent)
  date = get_and_update_date(sent)

  artists_from_mood = filter_data('mood', mood, data)

  if(len(genre)>0):
    artists_from_mood = filter_data('genre', genre, artists_from_mood)

  if(len(date)>0):
    artists_from_mood = filter_data('date', date, artists_from_mood)

  if(len(artists_from_mood)>0):
    track_selected = random.choice(artists_from_mood)

    result = '"' + track_selected['artist'] + '"'

    default_answer = answers['Artist from mood question'][0]

    response = default_answer.replace('*', mood)
    response = response.replace('_', result)
  else:
    response = ""

  return response

#GET SONG/ARTIST FROM DATE

In [52]:
def get_song_from_date(sent):
  date = get_and_update_date(sent)
  genre = get_and_update_genre(sent)

  tracks_from_date = filter_data('date', date, data)

  if(len(genre)>0):
    tracks_from_date = filter_data('genre', genre, tracks_from_date)

  if(len(tracks_from_date)>0):
    track_selected = random.choice(tracks_from_date)

    artist = '"' + track_selected['artist'] + '"'
    result = '"' + track_selected['track'] + '"'

    default_answer = answers['Song from date question'][0]

    response = default_answer.replace('*', date)
    response = response.replace('+', artist)
    response = response.replace('_', result)
  else:
    response = ""

  return response

def get_artist_from_date(sent):
  date = get_and_update_date(sent)
  genre = get_and_update_genre(sent)

  tracks_from_date = filter_data('date', date, data)

  if(len(genre)>0):
    tracks_from_date = filter_data('genre', genre, tracks_from_date)

  if(len(tracks_from_date)>0):
    track_selected = random.choice(tracks_from_date)

    result = '"' + track_selected['artist'] + '"'

    default_answer = answers['Artist from date question'][0]

    response = default_answer.replace('*', date)
    response = response.replace('_', result)
  else:
    response = ""

  return response

#GET SIMILAR ARTIST

In [53]:
def get_similar_artist(sent):
  artist = get_artist_from_sentence(sent)

  genre = get_genre_from_artist(artist)

  if(len(genre)>0):
    update_dialog_history(PREVIOUS_GENRE, genre)
  else:
    genre = get_dialog_history(PREVIOUS_GENRE)

  date = get_and_update_date(sent)
  mood = get_and_update_mood(sent)

  artists_from_genre = filter_data('genre', genre, data)

  if(len(date)>0):
    artists_from_genre = filter_data('date', date, artists_from_genre)

  if(len(mood)>0):
    artists_from_genre = filter_data('mood', mood, artists_from_genre)

  if(len(artists_from_genre)>0):
    artist_selected = random.choice(artists_from_genre)

    result = '"' + artist_selected['artist'] + '"'

    default_answer = answers['Similar artist question'][0]

    response = default_answer.replace('*', artist)
    response = response.replace('_', result)
  else:
    response = ""

  return response

#NOT VALID INPUT

In [54]:
def get_not_valid():
  default_answer = answers['Not valid'][0]
  return default_answer

#CHAT BOT

In [55]:
def chat_bot():

  reset_dialog_history() #to reset the dialog history each time you execute the chat_bot

  status = True
  
  print("Hello, I'm a music-recommender chatbot. Tell me what are you looking for and I will search in my music library. If you want to finish the conversation, just say "'"Goodbye."'"")

  while status:

    input_text = input("\nUser text: ")
    output_tag = predict_tag(input_text)

    print("\n")
    
    if(output_tag == "Conventional-closing"):
      status = False
      print("Goodbye!")

    elif(output_tag == "Conventional-opening"):
      print("Hello, tell me what genre of music you are into or your current mood.")


    else:
      output_tag = get_type_of_sentence(input_text)
      print("(Tag: " + output_tag + ")")

      print("\n--BOT ANSWER--\n")

      if(output_tag == question_labels[TOP_ARTISTS_FROM_GENRE]):
        print(get_artists_from_genre(input_text))

      if(output_tag == question_labels[TOP_TRACKS_FROM_GENRE]):
        print(get_tracks_from_genre(input_text))

      if(output_tag == question_labels[SONG_FROM_MOOD]):
        print(get_song_from_mood(input_text))

      if(output_tag == question_labels[ARTIST_FROM_MOOD]):
        print(get_artist_from_mood(input_text))

      if(output_tag == question_labels[SONG_FROM_DATE]):
        print(get_song_from_date(input_text))

      if(output_tag == question_labels[ARTIST_FROM_DATE]):
        print(get_artist_from_date(input_text))

      if(output_tag == question_labels[SIMILAR_ARTIST]):
        print(get_similar_artist(input_text))

      if(output_tag == question_labels[NOT_VALID]):
        print(get_not_valid())

      print_keyword_from_sentences(input_text)
      print_dialog_history()



In [56]:
#RUN THIS CODE IN ORDER TO START TALKING TO THE AGENT

chat_bot()

Hello, I'm a music-recommender chatbot. Tell me what are you looking for and I will search in my music library. If you want to finish the conversation, just say "Goodbye."

User text: do you know any rap song?
1/1 [==============================] - 3s 3s/step


(Score: 0.6113147)
(Tag: Top-tracks in genre question)

--BOT ANSWER--

The top-tracks in rap are:

1- THat Part
2- Juicy
3- If I Ruled the World (Imagine That)
4- Money Longer
5- In Da Club
6- 4r Da Squaw
7- Grindin'
8- Put It On
9- Night Job
10- Thug

--Keywords from CURRENT sentence--
Genre: rap
Mood: 
Date: 
Artist: 

--Dialog history--
Previous genre: rap
Previous mood: 
Previous date: 

User text: Do you know any song from the 80s?
1/1 [==============================] - 0s 76ms/step


(Score: 0.6786526)
(Tag: Artist from date question)

--BOT ANSWER--

You should listen to "50 Cent", an artist from 1980.

--Keywords from CURRENT sentence--
Genre: 
Mood: 
Date: 1980
Artist: 

--Dialog history--
Previous genre: rap
Previous 